<a href="https://colab.research.google.com/github/sahil3Vedi/Soccer-Prediction/blob/main/Odds_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Dependencies**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# importing dependencies
import os
import pandas as pd
import numpy as np
from random import randint
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import math

**Loading Dataframes having 2019 & 2020 Match Results for the League**

In [3]:
DATA_DIR = 'drive/My Drive/Odds'

BARCLAYS_PATHS = ['BPL Matches 2019.csv','BPL Matches 2020.csv']
LALIGA_PATHS = ['Spain 2019.csv','Spain 2020.csv']
SERIEA_PATHS = ['Italy 2019.csv','Italy 2020.csv']
LIGUE1_PATHS = ['France 2019.csv','France 2020.csv']
BUNDESLIGA_PATHS = ['Germany 2019.csv','Germany 2020.csv']
BELGIUM_PATHS = ['Belgium 2019.csv','Belgium 2020.csv']
SUPERLIG_PATHS = []
SCOTTISHLEAGUE_PATHS = ['Scottland 2019.csv','Scottland 2020.csv']
NETHERLANDS_PATHS = ['Netherlands 2019.csv','Netherlands 2020.csv']

LEAGUE_PATHS = NETHERLANDS_PATHS
MATCHES_2019_FILE = LEAGUE_PATHS[0]
MATCHES_2019_PATH = os.path.join(DATA_DIR,MATCHES_2019_FILE)
matches_2019 = pd.read_csv(MATCHES_2019_PATH)

MATCHES_2020_FILE = LEAGUE_PATHS[1]
MATCHES_2020_PATH = os.path.join(DATA_DIR,MATCHES_2020_FILE)
matches_2020 = pd.read_csv(MATCHES_2020_PATH)

**Predicting Outcomes for a Given Match using Poission Distribution**

In [4]:
def poissonify(u,x):
  return (math.exp(-u)*pow(u,x))/math.factorial(x)

def generatePoissonTable(home_exp,away_exp,home_team,away_team):
  num_goals = [0,1,2,3,4,5]
  home_distribution = [poissonify(home_exp,x) for x in num_goals]
  away_distribution = [poissonify(away_exp,x) for x in num_goals]
  #print("\n")
  #print(home_distribution)
  #print(away_distribution)
  #print("\n")
  distribution_df = {
      'Number of Goals':[str(home_team + " goals"),"0","1","2","3","4","5"],
      str(away_team + " goals"): ["Probability",home_distribution[0],home_distribution[1],home_distribution[2],home_distribution[3],home_distribution[4],home_distribution[5]],
      "0": [away_distribution[0],away_distribution[0]*home_distribution[0],away_distribution[0]*home_distribution[1],away_distribution[0]*home_distribution[2],away_distribution[0]*home_distribution[3],away_distribution[0]*home_distribution[4],away_distribution[0]*home_distribution[5]],
      "1": [away_distribution[1],away_distribution[1]*home_distribution[0],away_distribution[1]*home_distribution[1],away_distribution[1]*home_distribution[2],away_distribution[1]*home_distribution[3],away_distribution[1]*home_distribution[4],away_distribution[1]*home_distribution[5]],
      "2": [away_distribution[2],away_distribution[2]*home_distribution[0],away_distribution[2]*home_distribution[1],away_distribution[2]*home_distribution[2],away_distribution[2]*home_distribution[3],away_distribution[2]*home_distribution[4],away_distribution[2]*home_distribution[5]],
      "3": [away_distribution[3],away_distribution[3]*home_distribution[0],away_distribution[3]*home_distribution[1],away_distribution[3]*home_distribution[2],away_distribution[3]*home_distribution[3],away_distribution[3]*home_distribution[4],away_distribution[3]*home_distribution[5]],
      "4": [away_distribution[4],away_distribution[4]*home_distribution[0],away_distribution[4]*home_distribution[1],away_distribution[4]*home_distribution[2],away_distribution[4]*home_distribution[3],away_distribution[4]*home_distribution[4],away_distribution[4]*home_distribution[5]],
      "5": [away_distribution[5],away_distribution[5]*home_distribution[0],away_distribution[5]*home_distribution[1],away_distribution[5]*home_distribution[2],away_distribution[5]*home_distribution[3],away_distribution[5]*home_distribution[4],away_distribution[5]*home_distribution[5]]
      }
  dist_df = pd.DataFrame(data=distribution_df)
  #print(dist_df)

  home_probability = 0
  away_probability = 0
  draw_probability = 0

  for h in num_goals:
    for a in num_goals:
      outcome_probability = home_distribution[h]*away_distribution[a]
      if (h>a):
        home_probability += outcome_probability
      elif (h<a):
        away_probability += outcome_probability
      else:
        draw_probability += outcome_probability

  print(home_team + " Winning Probability: " + str(home_probability))
  print(away_team + " Winning Probability: " + str(away_probability))
  print("Draw Probability: " + str(draw_probability))
  print("\n")
  print(home_team + " Winning Odds: " + str(1/home_probability))
  print(away_team + " Winning Odds: " + str(1/away_probability))
  print("Draw Odds: " + str(1/draw_probability))
  print("\n")
  print("HX Odds: " + str(1/(home_probability+draw_probability)))
  print("XA Odds: " + str(1/(away_probability+draw_probability)))
  print("HA Odds: " + str(1/(away_probability+home_probability)))


  return (dist_df)

In [5]:
def analyseMatchUp(home_team,away_team):
  print("Home Team: " + home_team)
  print("Away Team: " + away_team)

  league_home_goals_19 = np.sum(matches_2019["FTHG"])
  league_away_goals_19 = np.sum(matches_2019["FTAG"])
  league_home_goals_20 = np.sum(matches_2020["FTHG"])
  league_away_goals_20 = np.sum(matches_2020["FTAG"])
  league_games_19 = len(matches_2019)
  league_games_20 = len(matches_2020)

  print("\nHome Goals Scored by All Teams in 2019: " + str(league_home_goals_19))
  print("Away Goals Scored by All Teams in 2019: " + str(league_away_goals_19))
  print("Total Games in 2019: " + str(league_games_19))
  print("\nHome Goals Scored by All Teams in 2020: " + str(league_home_goals_20))
  print("Away Goals Scored by All Teams in 2020: " + str(league_away_goals_20))
  print("Total Games in 2020: " + str(league_games_20))

  avg_home_goals = (league_home_goals_19 + league_home_goals_20)/(league_games_19 + league_games_20)
  avg_away_goals = (league_away_goals_19 + league_away_goals_20)/(league_games_19 + league_games_20)
  print("\nAverage Home Goals per Match: " + str(avg_home_goals))
  print("Average Away Goals per Match: " + str(avg_away_goals))

  home_found = 0
  home_team_home_performance = []
  for match_index in range(len(matches_2019)):
    if (matches_2019['HomeTeam'][match_index]==home_team):
      home_found = 1
      home_team_home_performance.append([matches_2019['FTHG'][match_index],matches_2019['FTAG'][match_index]])
  for match_index in range(len(matches_2020)):
    if (matches_2020['HomeTeam'][match_index]==home_team):
      home_found = 1
      home_team_home_performance.append([matches_2020['FTHG'][match_index],matches_2020['FTAG'][match_index]])
  if (home_found==0):
    print("\nPlease Check Name of Home Team")
    exit
    
  away_found = 0
  away_team_away_performance = []
  for match_index in range(len(matches_2019)):
    if (matches_2019['AwayTeam'][match_index]==away_team):
      away_found = 1
      away_team_away_performance.append([matches_2019['FTHG'][match_index],matches_2019['FTAG'][match_index]])
  for match_index in range(len(matches_2020)):
    if (matches_2020['AwayTeam'][match_index]==away_team):
      away_found = 1
      away_team_away_performance.append([matches_2020['FTHG'][match_index],matches_2020['FTAG'][match_index]])
  if (away_found==0):
    print("Please Check Name of Away Team")
    exit

  average_home_team_scored = np.average(np.asarray([a[0] for a in home_team_home_performance]))
  average_home_team_conceded = np.average(np.asarray([a[1] for a in home_team_home_performance]))
  average_away_team_scored = np.average(np.asarray([a[1] for a in away_team_away_performance]))
  average_away_team_conceded = np.average(np.asarray([a[0] for a in away_team_away_performance]))

  print("\nAverage Home Goals Scored by " + home_team + ": " + str(average_home_team_scored))
  print("Average Home Goals Conceded by " + home_team + ": " + str(average_home_team_conceded))
  print("Average Away Goals Scored by " + away_team + ": " + str(average_away_team_scored))
  print("Average Away Goals Conceded by " + away_team + ": " + str(average_away_team_conceded))

  home_team_attack_strength = average_home_team_scored/avg_home_goals
  away_team_attack_strength = average_away_team_scored/avg_away_goals
  home_team_defense_strength = average_home_team_conceded/avg_away_goals
  away_team_defense_strength = average_away_team_conceded/avg_home_goals

  print("\n" + home_team + " Attack Strength: " + str(home_team_attack_strength))
  print(away_team + " Attack Strength: " + str(away_team_attack_strength))
  print(home_team + " Defense Strength: " + str(home_team_defense_strength))
  print(away_team + " Defense Strength: " + str(away_team_defense_strength))

  home_team_goal_expectancy = home_team_attack_strength * away_team_defense_strength * avg_home_goals
  away_team_goal_expectancy = away_team_attack_strength * home_team_defense_strength * avg_away_goals

  print("\n" + home_team + " expected goals: " + str(home_team_goal_expectancy))
  print(away_team + " expected goals: " + str(away_team_goal_expectancy))

  return generatePoissonTable(home_team_goal_expectancy,away_team_goal_expectancy,home_team,away_team)

In [6]:
spread1 = analyseMatchUp('VVV Venlo','Heerenveen')

Home Team: VVV Venlo
Away Team: Heerenveen

Home Goals Scored by All Teams in 2019: 436
Away Goals Scored by All Teams in 2019: 279
Total Games in 2019: 232

Home Goals Scored by All Teams in 2020: 224
Away Goals Scored by All Teams in 2020: 212
Total Games in 2020: 135

Average Home Goals per Match: 1.7983651226158037
Average Away Goals per Match: 1.337874659400545

Average Home Goals Scored by VVV Venlo: 1.1428571428571428
Average Home Goals Conceded by VVV Venlo: 2.238095238095238
Average Away Goals Scored by Heerenveen: 1.8
Average Away Goals Conceded by Heerenveen: 1.85

VVV Venlo Attack Strength: 0.6354978354978355
Heerenveen Attack Strength: 1.345417515274949
VVV Venlo Defense Strength: 1.6728736301037725
Heerenveen Defense Strength: 1.0287121212121213

VVV Venlo expected goals: 1.1756709956709956
Heerenveen expected goals: 3.0111725341867905
VVV Venlo Winning Probability: 0.11568345802460703
Heerenveen Winning Probability: 0.6579838729206129
Draw Probability: 0.1400452594346475

In [7]:
spread1

,Number of Goals,Heerenveen goals,0,1,2,3,4,5
0,VVV Venlo goals,Probability,0.049234,0.148252,0.223206,0.224037,0.168654,0.101569
1,0,0.308612,0.015194,0.045752,0.068884,0.069141,0.052049,0.031345
2,1,0.362826,0.017863,0.053790,0.080985,0.081287,0.061192,0.036852
3,2,0.213282,0.010501,0.031619,0.047606,0.047783,0.035971,0.021663
4,3,0.0835832,0.004115,0.012391,0.018656,0.018726,0.014097,0.008489
5,4,0.0245666,0.001210,0.003642,0.005483,0.005504,0.004143,0.002495
6,5,0.00577644,0.000284,0.000856,0.001289,0.001294,0.000974,0.000587
